# Calculating timespent 


## Timespent on a dataset or on a subject


### Student activity day by day

Unit information on log files produced by the game is the **record**. 


#### Group activity of a student by day by dataset
First step for computing stats consists of building a table called `student_day` (complete script to generate the table is stored in file `db/stats/day.py`) using day as the unit of time to delimit day of play and sort the corresponding records. 

Each record is qualified by many parameters but to compute timespent and activity by day by dataset or by subject we only need to group by thoses 4 values:

- `student`
- `dataset`
- `subject`
- `day`
- `unixTime`

Process then simply consists on grouping the `records` of one **student** for one **dataset** by **day** using the unixTime and aggregate its belonging records into a list.

For *assessments_\** datasets as records are ordered by assessementEndTime and can be over different days we group records using assessmentEndTime as additionnal filter. Then we take first and last unixtime of the group of records and compute timespent


**Example:**
    
> db.student_day.findOne()


returns:

```json
{
	"classroom" : 1,
	"dataset" : "gapfill_lang",
	"day" : "2018-11-23",
	"student" : 111,
	"subject" : "letters",
	"records": [
        {
			"tag" : "lame",
			"value" : "lame",
			"unixTime" : ISODate("2018-11-23T10:52:18Z"),
			"score" : 1,
			"stimulus" : "mét",
			"target" : "lame",
			"click" : 0,
			"chapter" : 15,
			"lesson" : null,
			"elapsedTime" : -1
		},
		{
			"tag" : "lame",
			"value" : "lame",
			"unixTime" : ISODate("2018-11-23T10:52:18Z"),
			"score" : 1,
			"stimulus" : "lema",
			"target" : "lame",
			"click" : 0,
			"chapter" : 15,
			"lesson" : null,
			"elapsedTime" : -1
		},
		{
			"tag" : "lame",
			"value" : "lame",
			"unixTime" : ISODate("2018-11-23T10:52:18Z"),
			"score" : 1,
			"stimulus" : "lame",
			"target" : "lame",
			"click" : 1,
			"chapter" : 15,
			"lesson" : null,
			"elapsedTime" : 2.230761
		},
		{
			"tag" : "vélo",
			"value" : "vélo",
			"unixTime" : ISODate("2018-11-23T10:52:26Z"),
			"score" : 0,
			"stimulus" : "mru",
			"target" : "vélo",
			"click" : 1,
			"chapter" : 2,
			"lesson" : null,
			"elapsedTime" : 2.20513
		},
		{
			"tag" : "vélo",
			"value" : "vélo",
			"unixTime" : ISODate("2018-11-23T10:52:26Z"),
			"score" : 1,
			"stimulus" : "vléo",
			"target" : "vélo",
			"click" : 0,
			"chapter" : 2,
			"lesson" : null,
			"elapsedTime" : -1
		},
		{
			"tag" : "vélo",
			"value" : "vélo",
			"unixTime" : ISODate("2018-11-23T10:52:26Z"),
			"score" : 1,
			"stimulus" : "vélo",
			"target" : "vélo",
			"click" : 1,
			"chapter" : 2,
			"lesson" : null,
			"elapsedTime" : 6.873475
		}
	]

}

```

Once we have this list of records belonging to the student on a dataset on a day, we can sort the list of records attached by `unixTime` information attached to any record.

So we can get the first and last unixTime considering it respectively as start and end of the day.


On `student_day` table, we calculate the timespent simply making the difference between endTime of the day  and start Time of the day.



```sql
# pseudo code in SQL
FROM RECORDS 
SELECT record
GROUP BY student,dataset,day
SORTED BY student, dataset and unixtime
SELECT FIRST unixTime LAST unixTime
CALCULATE timespent (LAST-FIRST)
INSERT into student_day
```           


```
# MongoDB aggregation equivalent

pipeline = [{
			"$sort": SON([("student", 1), ("unixTime", 1)])
		},
		{
			"$group": {
				"_id": {
					"classroom": "$classroom",
					"student": "$student",
					"group": "$group",
					"dataset": "$dataset",
					"day": "$day",
					"subject": "$subject",
				},
				"start": {"$first": "$unixTime"},
				"end": {"$last": "$unixTime"},
				"nb_records": {"$sum": 1},
				"records": {"$push": "$$ROOT"},
				
			}
		}
        {
			"$out": "student_day"
		}]
db.records.aggregate(pipeline)
```

**Result**
    


```
{
	"classroom" : 1,
	"dataset" : "numbers",
	"day" : "2019-02-25",
	"group" : "r/m",
	"nb_records" : 198,
	"nb_sequences" : 1,
	"sequences" : [
		[
			ISODate("2019-02-25T08:54:29Z"),
			ISODate("2019-02-25T09:17:16Z")
		]
	],
	"start" : ISODate("2019-02-25T08:54:29Z"),
	"start_date" : "2019-02-25 09:54:29",
    "end" : ISODate("2019-02-25T09:17:16Z"),
	"end_date" : "2019-02-25 10:17:16",
	"student" : 111,
	"subject" : "numbers",
	"timespent_min" : 22.783333333333335,
	"timespent_sec" : 1367
    "records":[]
}

```

### Controling timespent by student by subject

The dashboard displays the timespent on the whole game for a **subject** for a **specific student**.

![](./screenshots/widget_activity.png)


The timespent exposed here is provided by the API 
and corresponds to the data loaded from the database 
and exposed throught an url that specify the student and the subject



Lets create a function to have acess to timespent for the student and the subject chosen:


In [7]:
import requests

API_URL = "https://research.ludoeducation.fr"

def get_student_timespent_on_subject(student, subject="numbers"):
    endpoint = "/activity/students/{}/subjects/{}".format(student, subject)
    uri = API_URL + endpoint
    #print(uri)
    r = requests.get(uri)
    data = r.json()
    if "data" in data:
        return data["data"][0]["timespent"]
    else:
        #if no data this means that we have no data for this student on this subject
        return None

If I want to check the timespent of student n° `153` on `numbers` I will use the previous function:

In [8]:
student = 153
subject= "numbers"
timespent = get_student_timespent_on_subject(student, subject)
print("Elève", student, timespent, subject )


Elève 153 00:24:50 numbers


Now I want to verify timespent for all the students on subject "numbers". Lets create a function that retrieve all the students available.


In [9]:
def get_student_list():
    '''Load student id'''
    endpoint = "/admin/students"
    uri = API_URL + endpoint
    #print(uri)
    r = requests.get(uri)
    data = r.json()
    return [s["_id"] for s in data["students"]if s["group"]!= "guest"]


In [11]:
for student in get_student_list():
    timespent = get_student_timespent_on_subject(student, "numbers")
    print(student, timespent)

3221 01:59:19
3741 00:00:23
3722 00:21:50
912 00:04:00
474 00:14:50
3244 00:03:13
1021 00:04:32
2013 00:15:54
3225 00:16:31
1822 00:14:36
4323 00:05:37
1234 00:01:20
471 00:14:46
3473 00:01:04
1922 00:05:16
3162 00:11:41
522 00:00:28
1683 00:01:13
3422 00:10:08
2741 00:13:02
3253 00:11:13
833 00:00:00
4212 00:17:09
3914 00:05:16
1123 00:17:03
29132 00:01:29
3812 00:11:06
3936 00:02:02
3353 00:02:33
972 00:01:03
3445 None
2941 00:18:14
1222 00:06:06
714 00:03:10
3274 00:17:56
723 00:03:01
1651 00:19:14
612 00:04:13
1031 00:16:35
924 00:03:42
311 00:14:18
29105 None
2631 00:05:20
1941 00:21:44
443 00:09:03
4521 00:14:26
1184 00:01:55
822 00:04:36
4344 00:16:01
533 00:04:00
2074 00:06:23
323 00:05:50
3323 00:01:44
4051 00:07:41
2644 None
2621 00:11:57
4451 23:51:02
4343 00:09:54
2071 00:05:24
1231 00:42:18
1023 00:15:03
3951 00:00:44
2624 None
3333 00:03:10
1144 00:12:07
2751 00:12:31
3842 00:03:20
2052 00:11:08
1426 None
652 00:08:12
123 00:24:21
3121 00:09:07
3072 00:05:44
524 00:00:00


1952 00:12:07
3281 00:10:40
3922 None
1751 00:41:09
2971 00:02:04
3223 00:06:39
623 00:08:46
771 00:12:18
3341 00:07:48
1622 00:02:22
1524 00:20:22
1454 00:13:18
2073 00:07:52
913 00:02:16
1823 00:05:01
1264 00:00:39
4421 00:08:59
3423 00:30:17
842 00:10:57
3322 00:01:20
141 00:12:37
2051 00:15:03
3863 00:49:27
1961 00:49:48
433 00:16:32
1966 None
3251 00:01:28
1741 00:03:57
2952 00:07:38
434 00:06:12
1133 00:12:48
2033 00:02:24
29133 00:00:11
3943 00:00:12
1464 00:15:00
122 00:21:43
663 00:00:00
2034 00:15:59
1181 00:03:38
2062 00:17:30
1041 00:01:27
543 00:05:26
744 00:16:17
3483 00:04:13
2712 00:15:52
2641 00:03:28
743 00:05:22
3163 00:17:06
3241 03:37:20
1424 00:02:01
422 00:42:34
472 00:13:06
29102 00:19:42
1814 00:22:16
3124 00:12:36
1212 00:28:32
1534 None
631 00:13:01
2022 00:14:38
3463 00:07:50
1831 00:04:30
1711 None
1034 00:15:05
29111 00:00:30
4012 00:00:00
3343 00:18:06
3362 00:02:15
3833 00:25:12
1632 00:19:26
1761 00:10:11
1821 00:13:27
1432 00:11:05
2962 00:05:05
783 00

In [12]:
import plotly.graph_objects as go

def build_histogram(subject="numbers"):
    x = get_student_list()
    y = [get_student_timespent_subject(s, subject) for s in x]

    # Use textposition='auto' for direct text
    fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
        )])
    fig.show()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

